In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


import matplotlib.patches as patches
from collections import Counter
import os
import shutil
import random
import xml.etree.ElementTree as ET
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset,DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split

import sys
import torch.optim as optim
from tqdm.notebook import tqdm
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
device = torch.device ("cuda:0" if torch.cuda.is_available () else "cpu")

In [3]:
path_annotations = "/content/drive/MyDrive/birds/birds/annotations/" 
images_dir = "/content/drive/MyDrive/birds/birds/"

path_annotations = "/home/costia/birds/annotations/" #"/home/costia/birds_project/detect/annotations/check/"
images_dir = "/home/costia/birds/"

In [15]:
labels_dict = {
    1: 'Eurasian_jay',
    2: 'great_spotted_woodpecker',
    3: 'greenfinch',
    4: 'blue_tit',
    5: 'Carduelis',
    6: 'common_redpoll',
    7: 'great_tit',
    8: 'bullfinch',
    9: 'Eurasian_siskin',
    10: 'Eurasian_tree_sparrow',
    11: 'hawfinch',
    12: 'willow_tit',
    13: 'Fieldfare',
    14: 'Common_chaffinch',
    15: 'Common_linnet'}
n_classes = len(labels_dict)  # number of classes
color_dict = {i:color/255 for i,color in enumerate(np.random.uniform(0, 255, size=(n_classes, 3)))}

In [5]:
def convert_2_yolo(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x_min, y_min, x_max, y_max = box
    x = (x_min + x_max)/2.0 - 1
    y = (y_min + y_max)/2.0 - 1
    w = x_max - x_min
    h = y_max - y_min
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x, y , w, h)

def xml_transform(xml_path, im_size = None):
    xml = ET.parse(xml_path)
    root = xml.getroot()
    img_name = root[1].text
    labels, boxes = [], [] 
    
    size = (float(root[4][0].text), float(root[4][1].text))
    #print("(w,h)", size, img_name) # test info
    for obj in root.iter('object'):
        label = obj[0].text
        if label == 'Common chaffinch':
            label = 'Common_chaffinch'
        try:
            label = [label == species for species in labels_dict.values()].index(True) + 1
        except ValueError:
            print(xml_path)
                
        labels.append(label)
        box = [int(coord.text) for coord in obj[4]]
        if im_size:
            dw = im_size[0]/(size[0])
            dh = im_size[1]/(size[1])
            box = [int(box[0]*dw), int(box[1]*dh), int(box[2]*dw), int(box[3]*dh)]
            box = convert_2_yolo(im_size, box)
        else:
            box = convert_2_yolo(size, box)
        boxes.append(box)
        
    target = {'boxes' : torch.as_tensor(boxes, dtype=torch.float32),
              'labels' : labels}
    return img_name, target



def count_classes(path_annotations):
    target_list = []
    xml_list = [os.path.join(path_annotations, file) for file in os.listdir(path_annotations)]
    for xml in xml_list:
        _, target = xml_transform(xml)
        target_list += target['labels']
    return Counter(target_list) 

In [8]:
def plot_detection(img, info, with_text=False, conf_lvl=0.3, swap=True,
                   color_dict=color_dict):
    fig, ax = plt.subplots()
    img = img.permute((1, 2, 0))

    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img.numpy() + mean

    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    img = np.clip(img, 0, 1)
    
    if swap:
        img = img.swapaxes(0,1)
        img = img.swapaxes(1,2)
        img = np.clip(img, 0, 1)
    ax.imshow(img)
    
    dh, dw, _ = img.shape
    
    flag = False
    if 'scores' in info.keys():
        mask = info['scores'] >= conf_lvl
        confidences = info['scores'][mask]
        boxes = info['boxes'][mask]
        labels = info['labels'][mask]
        
        flag = True
    else:
        
        boxes = info['boxes']
        labels = info['labels']
    
    for i, box in enumerate(boxes):
        '''
        x_min, y_min, x_max, y_max = box
        height = y_max - y_min
        width = x_max - x_min
        '''
        x_center, y_center, width, height = box
        x_min = x_center - width/2
        y_min = y_center - height/2
        width = int(width*dw)
        height = int(height*dh)
        x_min = int(x_min*dw)
        y_min = int(y_min*dh)
        
        label = labels_dict [int(labels[i])]
        color = color_dict[int(labels[i])]
        rect = patches.Rectangle((x_min, y_min), width, height,
                                 linewidth=2, edgecolor=color,
                                 facecolor='none')
        ax.add_patch(rect)
        if with_text:
            if flag:
                text = '{} {:.0f}%'.format(label, confidences[i]*100)
            else:
                text = '{}'.format(label)
            ax.text(x_min, y_min, text,
                    bbox=dict(facecolor='white', alpha=0.5))
    plt.show()

In [86]:
def dir_check(path = path_annotations, addition = '_YOLO/'):
    new_path = path[:-1] + addition 
    directory = os.path.isdir(new_path)
    if not directory:
        os.makedirs(new_path)
        print("created folder : ", new_path)
    else:
        print(directory, "folder already exists.")
    return new_path

        
def xml_transform_to_YOLO(xml_path, im_size = None, images_dir = images_dir ):
    xml = ET.parse(xml_path)
    root = xml.getroot()
    img_name = root[1].text
    labels, boxes = [], [] 
    
    size = (float(root[4][0].text), float(root[4][1].text))
    #print("(w,h)", size, img_name) # test info
    for obj in root.iter('object'):
        label = obj[0].text
        if label == 'Common chaffinch':
            label = 'Common_chaffinch'
        try:
            label = [label == species for species in labels_dict.values()].index(True) + 1
        except ValueError:
            print(xml_path)
                
        labels.append(label)
        box = [int(coord.text) for coord in obj[4]]
        if im_size:
            dw = im_size[0]/(size[0])
            dh = im_size[1]/(size[1])
            box = [int(box[0]*dw), int(box[1]*dh), int(box[2]*dw), int(box[3]*dh)]
            box = convert_2_yolo(im_size, box)
        else:
            box = convert_2_yolo(size, box)
        boxes.append(box)
    annotations = ["{} {:.3f} {:.3f} {:.3f} {:.3f}".format(cls, bxs[0], bxs[1], bxs[2], bxs[3])
                   for cls, bxs in zip(labels , boxes)]
    
    save_file_name = xml_path.replace("xml", "txt").replace("annotations", "annotations_YOLO")   
      
    print("\n".join(annotations), file = open(save_file_name, "w"))
    imgs_path.append(os.path.join(images_dir, img_name))
    ann_path.append(save_file_name)


def copy_files(files_list, dst):
    for src in files_list:
        shutil.copy(src, dst)
    

In [78]:
# list of paths for all annotations files
path_list = [os.path.join(path_annotations, path) for path in os.listdir(path_annotations)]
#check if annotations YOLO is created
dir_check(path_annotations)
imgs_path = []
ann_path = []
for xml_path in path_list:
    xml_transform_to_YOLO(xml_path)

True folder already exists.


In [80]:
# Split the dataset into train-valid-test splits in 0.8 / 0.1 / 0.1 ratio 
train_imgs, val_imgs, train_annots, val_annots = train_test_split(
    imgs_path, ann_path, test_size = 0.2, random_state = 756)
val_imgs, test_imgs, val_annots, test_annots = train_test_split(
    val_imgs, val_annots, test_size = 0.5, random_state = 756)

In [91]:
tr_an = dir_check(path = path_annotations, addition = '_YOLO/train/')
test_an = dir_check(path = path_annotations, addition = '_YOLO/test/')
val_an = dir_check(path = path_annotations, addition = '_YOLO/val/')
im_tr = dir_check(path = images_dir, addition = '_YOLO/train/')
im_test = dir_check(path = images_dir, addition = '_YOLO/test/')
im_val = dir_check(path = images_dir, addition = '_YOLO/val/')


True folder already exists.
True folder already exists.
True folder already exists.
True folder already exists.
True folder already exists.
True folder already exists.


In [92]:
dirs = [tr_an, test_an, val_an,
        im_tr, im_test, im_val]
paths = [train_annots, test_annots, val_annots,
        train_imgs, test_imgs, val_imgs]

for files, folder in zip(paths, dirs):
    copy_files(files, folder)

In [94]:
labels_dict.values()

dict_values(['Eurasian_jay', 'great_spotted_woodpecker', 'greenfinch', 'blue_tit', 'Carduelis', 'common_redpoll', 'great_tit', 'bullfinch', 'Eurasian_siskin', 'Eurasian_tree_sparrow', 'hawfinch', 'willow_tit', 'Fieldfare', 'Common_chaffinch', 'Common_linnet'])

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git #clone repo
!cd yolov5/
!pip install -r requirements.txt #install dependencies

In [ ]:
python train.py --img 640 --cfg yolov5s.yaml --hyp hyp.scratch-med.yaml --batch 8 --epochs 100 --data birds_data.yaml --weights yolov5s.pt --workers 24 --name yolo_birds


In [101]:
from glob import glob
glob(images_dir + "/*/", recursive = True)

['/home/costia/birds/annotations_YOLO/', '/home/costia/birds/annotations/']

In [103]:
from pathlib import Path

# prefix components:
space =  '    '
branch = '│   '
# pointers:
tee =    '├── '
last =   '└── '


def tree(dir_path: Path, prefix: str=''):
    """A recursive generator, given a directory Path object
    will yield a visual tree structure line by line
    with each line prefixed by the same characters
    """    
    contents = list(dir_path.iterdir())
    # contents each get pointers that are ├── with a final └── :
    pointers = [tee] * (len(contents) - 1) + [last]
    for pointer, path in zip(pointers, contents):
        yield prefix + pointer + path.name
        if path.is_dir(): # extend the prefix and recurse:
            extension = branch if pointer == tee else space 
            # i.e. space because last, └── , above so no more |
            yield from tree(path, prefix=prefix+extension)
            


In [115]:
for line in tree(Path('/home/costia/yolov5/runs/train')):
    print(line)

├── yolo_birds2
│   ├── labels_correlogram.jpg
│   ├── events.out.tfevents.1662755041.costia-Z68XP-UD3.293160.0
│   ├── hyp.yaml
│   ├── weights
│   ├── opt.yaml
│   └── labels.jpg
├── yolo_birds4
│   ├── labels_correlogram.jpg
│   ├── events.out.tfevents.1662840617.costia-Z68XP-UD3.428090.0
│   ├── hyp.yaml
│   ├── weights
│   ├── opt.yaml
│   ├── labels.jpg
│   ├── train_batch0.jpg
│   ├── train_batch1.jpg
│   └── train_batch2.jpg
├── yolo_birds
│   ├── hyp.yaml
│   ├── weights
│   ├── opt.yaml
│   └── events.out.tfevents.1662754888.costia-Z68XP-UD3.292815.0
├── yolo_birds5
│   ├── val_batch0_pred.jpg
│   ├── labels_correlogram.jpg
│   ├── confusion_matrix.png
│   ├── events.out.tfevents.1662878816.costia-Z68XP-UD3.485461.0
│   ├── PR_curve.png
│   ├── results.png
│   ├── val_batch2_pred.jpg
│   ├── hyp.yaml
│   ├── results.csv
│   ├── weights
│   │   ├── best.pt
│   │   └── last.pt
│   ├── val_batch2_labels.jpg
│   ├── opt.yaml
│   ├── R_curve.png
│   ├── labels.jpg
│   ├── train_ba

In [109]:
os.listdir('/home/costia/yolov5/runs/train/yolo_birds5')

['val_batch0_pred.jpg',
 'labels_correlogram.jpg',
 'confusion_matrix.png',
 'events.out.tfevents.1662878816.costia-Z68XP-UD3.485461.0',
 'PR_curve.png',
 'results.png',
 'val_batch2_pred.jpg',
 'hyp.yaml',
 'results.csv',
 'weights',
 'val_batch2_labels.jpg',
 'opt.yaml',
 'R_curve.png',
 'labels.jpg',
 'train_batch0.jpg',
 'train_batch1.jpg',
 'val_batch1_labels.jpg',
 'val_batch1_pred.jpg',
 'val_batch0_labels.jpg',
 'F1_curve.png',
 'P_curve.png',
 'train_batch2.jpg']

In [113]:
glob('/home/costia/yolov5/runs/train/yolo_birds5/', recursive = True)

['/home/costia/yolov5/runs/train/yolo_birds5/']